In [ ]:
import pandas as pd  # مكتبة معالجة البيانات
import json          # مكتبة التعامل مع ملفات الإعدادات
import logging       # مكتبة تسجيل الأحداث (بديلة لـ print)
from pathlib import Path  # مكتبة التعامل مع مسارات الملفات بشكل حديث

# 1. إعداد نظام التسجيل (Logging)
# هذا الجزء يسجل كل ما يحدث في البرنامج في ملف خارجي وعلى الشاشة
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("سجل_التنظيف.log", encoding='utf-8'), # حفظ السجل في ملف
        logging.StreamHandler() # عرض السجل في كونسول بايثون
    ]
)

class منظف_البيانات:
    """كلاس (فئة) لتنظيف البيانات آلياً بناءً على إعدادات ملف JSON."""

    def __init__(self, ملف_الإعدادات: str):
        # استخدام Pathlib للتعامل مع المسارات (أمر احترافي )
        self.المسار_الأساسي = Path(__file__).parent.parent
        self.مسار_الإعدادات = self.المسار_الأساسي / "config" / ملف_الإعدادات
        self.الإعدادات = self.تحميل_الإعدادات()
        self.البيانات = None

    def تحميل_الإعدادات(self):
        """تقرأ الإعدادات من ملف JSON وتخزنها في البرنامج."""
        try:
            with open(self.مسار_الإعدادات, 'r', encoding='utf-8') as f:
                logging.info(f"تم تحميل الإعدادات من: {self.مسار_الإعدادات}")
                return json.load(f)
        except Exception as e:
            logging.error(f"خطأ في تحميل الإعدادات: {e}")
            return None

    def تحميل_البيانات(self, اسم_الملف: str):
        """تقرأ ملف الـ CSV المراد تنظيفه."""
        مسار_البيانات = self.المسار_الأساسي / "data" / اسم_الملف
        try:
            self.البيانات = pd.read_csv(مسار_البيانات)
            logging.info(f"تم تحميل البيانات بنجاح. عدد الأسطر: {len(self.البيانات)}")
        except Exception as e:
            logging.error(f"خطأ في تحميل ملف البيانات: {e}")

    def حذف_القيم_المفقودة(self):
        """تحذف الأسطر التي تحتوي على خلايا فارغة."""
        if self.الإعدادات.get("drop_na"):
            العدد_قبل = self.البيانات.shape[0]
            self.البيانات = self.البيانات.dropna()
            logging.info(f"تم حذف القيم الفارغة. الأسطر قبل: {العدد_قبل}، بعد: {self.البيانات.shape[0]}")

    def حذف_التكرارات(self):
        """تحذف الأسطر المتكررة تماماً."""
        if self.الإعدادات.get("drop_duplicates"):
            العدد_قبل = self.البيانات.shape[0]
            self.البيانات = self.البيانات.drop_duplicates()
            logging.info(f"تم حذف التكرارات. الأسطر قبل: {العدد_قبل}، بعد: {self.البيانات.shape[0]}")

    def تصحيح_أنواع_البيانات(self):
        """تغير نوع البيانات (مثلاً من نص إلى رقم) بناءً على ملف الإعدادات."""
        التصحيحات = self.الإعدادات.get("data_type_corrections", {})
        for العمود, النوع in التصحيحات.items():
            if العمود in self.البيانات.columns:
                try:
                    self.البيانات[العمود] = self.البيانات[العمود].astype(النوع)
                    logging.info(f"تم تغيير نوع عمود '{العمود}' إلى {النوع}.")
                except Exception as e:
                    logging.warning(f"فشل تغيير نوع العمود '{العمود}': {e}")

    def حذف_الأعمدة(self):
        """تحذف الأعمدة التي حددناها كأعمدة غير ضرورية."""
        أعمدة_للحذف = self.الإعدادات.get("remove_columns", [])
        الأعمدة_الموجودة = [عمود for عمود in أعمدة_للحذف if عمود in self.البيانات.columns]
        if الأعمدة_الموجودة:
            self.البيانات = self.البيانات.drop(columns=الأعمدة_الموجودة)
            logging.info(f"تم حذف الأعمدة التالية: {الأعمدة_الموجودة}")

    def حفظ_البيانات(self, اسم_ملف_المخرجات: str):
        """تحفظ الملف النهائي المنظف في مجلد data."""
        مسار_الحفظ = self.المسار_الأساسي / "data" / اسم_ملف_المخرجات
        try:
            self.البيانات.to_csv(مسار_الحفظ, index=False)
            logging.info(f"تم حفظ البيانات المنظفة في: {مسار_الحفظ}")
        except Exception as e:
            logging.error(f"خطأ أثناء الحفظ: {e}")

    def تشغيل(self, ملف_المدخلات: str, ملف_المخرجات: str):
        """تشغيل 'خط الإنتاج' بالترتيب."""
        logging.info("بدء عملية التنظيف...")
        self.تحميل_البيانات(ملف_المدخلات)

        if self.البيانات is not None:
            self.حذف_القيم_المفقودة()
            self.حذف_التكرارات()
            self.تصحيح_أنواع_البيانات()
            self.حذف_الأعمدة()
            self.حفظ_البيانات(ملف_المخرجات)
            logging.info("تمت العملية بنجاح!")

# تشغيل البرنامج فعلياً
if __name__ == "__main__":
    المنظف = منظف_البيانات("cleaning_config.json")
    المنظف.تشغيل("my_data.csv", "bereinigte_daten.csv")


البرمجة كائنية التوجه (OOP):
الشرح: بدلاً من كتابة كود متسلسل يصعب تعديله، قمت بإنشاء كلاس DatenBereiniger. هذا الكلاس يجمع كل وظائف التنظيف في مكان واحد، مما يجعل الكود قابلاً لإعادة


الاستخدام مع أي مشروع آخر بمجرد تغيير ملف الإعدادات.

استخدام مكتبة Pathlib:


الشرح: استخدمت Pathlib لإدارة مسارات الملفات بدلاً من مكتبة os القديمة. هذا يضمن أن البرنامج سيعمل على ويندوز، ماك، ولينكس بدون مشاكل في "الشرطات" الخاصة بالمسارات (Slashes).

تقسيم الوظائف (Refactoring):

الشرح: قمت بتقسيم العملية الكبيرة إلى وظائف (Methods) صغيرة ومستقلة (entferne_fehlwerte لحذف القيم الفارغة، entferne_duplikate للتكرارات، إلخ). هذا يسهل عملية "تنقيح الكود" (Debugging) واختبار كل جزء على حدة.

نظام التسجيل (Logging):

الشرح: استبدلت أوامر print بنظام الـ logging. هذا النظام يسجل الوقت والتاريخ ونوع الرسالة (معلومات INFO أو خطأ ERROR) في ملف سجل. هذا هو الأسلوب المتبع في بناء البرامج الحقيقية لمراقبة أدائها.



لماذا تستخدم ملف JSON للإعدادات؟"
إجابتك: "لأن ذلك يفصل 'المنطق' (Logic) عن 'البيانات'. إذا أراد المستخدم تغيير الأعمدة التي يحذفها، فإنه يعدل ملف الـ JSON فقط ولا يحتاج للمس كود البايثون أبداً، وهذا أكثر أماناً واحترافية."

